# Create a local information system of a Thin-Section

### TEST files

requirements:

- Python 3
- GDAL

## 1. Input and settings

In this section we collect input data and global settings.
We assume that we have a raster file of a Thin-section digitalized in any way and we have defined a polygon feature that overlay minerals or other elements of the thin-section.

The poligon feature layer must be in a GIS-readable format.
We use gadl2tiles with no-crs definition

In [1]:
# verify Python version
import sys
if sys.version_info[0] < 3:
    raise Exception("Python 3 not detected.")
else :
    print("Python 3 detected. OK.")

# verify GDAL version
try:
    from osgeo import gdal
    print(f"GDAL version: {gdal.__version__}")
except ImportError:
    raise ImportError("GDAL not detected.")

Python 3 detected. OK.
GDAL version: 3.8.5


# 1. Import files

In this section we use `upload_files_widgets` from `lis_function.py` to import data and files usinng widgets.

Next, store the file in a local subfolder with the same name of "project name" gived by the user.

In [2]:
# Input widgets

from lis_functions import upload_files_widgets
Shapefile_selector, Jpgfile_selector, projectname = upload_files_widgets()
# Shapefile_selector, Jpgfile_selector, projectname, EPSGcode = upload_files_widgets()


Label(value='Select ALL the shapefile files [.shp, .dbf, .shx, .cpg]')

FileUpload(value=(), accept='.shp, .dbf, .shx, .cpg', description='Shapefile polygon layer', multiple=True)

Label(value='Select the Image file')

FileUpload(value=(), description='Image')

Label(value='Select the project name')

Text(value='', description='Name:', placeholder='NAME')

Label(value='Note: ouptut folder will be created in the same directory as the input files')

In [3]:
from lis_functions import save_to_temp_dir
import os

# Save file to a temporary directory

files_path = save_to_temp_dir(Shapefile_selector, Jpgfile_selector, projectname.value)

# Run gdal2tiles


# 2. Create Raster tiles using Gdal

Using a subprocess, we run the function `run_gdal2tiles`, that launch the gdal2tiles procedure in terminal.

By default a simple webviewer is written and stored in openalayer.html file inside the project folder.

In [4]:
from lis_functions import run_gdal2tiles

# Create raster tiles using gdal2tiles
# for customization see function in lis_functions.py

run_gdal2tiles(files_path['Raster_path'], projectname.value)


WARNING! - Directory already exists, Files will be overwrited! Continue? (Y).


Generating Base Tiles:


0...10...20...30...40...50...60...70...80...90...100 - done.
0.

Generating Overview Tiles:


..10...20...30...40...50...60...70...80...90...100 - done.
Tiles stored in: test_PAL11


## TODO

- For further implementation read `tilemapsoruce.xml` file generated after gdal2tiles subprocess



# 3. Create LIS map viewer of data

## 3.1 Convert SHP to geojson using GDAL ogr2ogr

Using a subprocess we convert in GeoJson the shapefiles provided


In [5]:


import subprocess
import os

def convert_shp_to_geojson(input_shp, output_geojson):
    """
    SHP to GeoJSON conversion using ogr2ogr.
    
    Args:
        input_shp (str): path to the input SHP file.
        output_geojson (str): path to the output GeoJSON file.
    """
    try:
        # gdal string command
        ogr2ogr_command = [
            'ogr2ogr',
            '-f', 'GeoJSON',  # output format
            output_geojson,   # output file
            input_shp         # input file
        ]
        
        # RUN
        subprocess.run(ogr2ogr_command, check=True)
        print(f'Conversion completed: {output_geojson}')
    except subprocess.CalledProcessError as e:
        print(f'{e}')

# Example
input_shp = files_path['Shp_file_path']

output_geojson = projectname.value + '/' + os.path.basename(input_shp) +'.geojson'

convert_shp_to_geojson(input_shp, output_geojson)

Conversion completed: test_PAL11/PAL11.shp.geojson


## 3.2 add GeoJson to defautl openalyers.html file

We access the openalyers.html defautl file generated by gdal2tiles and add some simple javascript code to overlay the previously created GeoJSON.

1. We use the `geojson_template.js` a JavaScript file prepared with the script code to be added to mapviewer, in the next cell simply we change the placeholder to the current GeoJSON file.
2. We create the javascript code and add it as a source script to the `openlayers.html` file generated earlier by the gdal2tiles subprocess


**NOTE**: For a correct map overlay in browser, we assume that the nominal reference system is EPSG:3857, the same one used by default by OpenLayers in the web view. If other SRSs are used, vector overlay to raster tiles is uncorrect.
This is a fictitious assumption just for running webviewer. It works only if Vector Layer and Raster Layer are alredy correct overlapped previously and was saved without SRS definition

In [6]:
from lis_functions import create_geojson_js, append_js_to_html

# STEP 1 create geojson.js file 
geojson_file_path = output_geojson
template_js_path = 'geojson_template.js'
destination_path = projectname.value + '/geojson.js'

create_geojson_js(geojson_file_path, template_js_path, destination_path)

# STEP 2 append the js file to the html file

# RUN
js_file_path = destination_path
html_file_path = projectname.value + '/' + 'openlayers.html'
new_htmlfile_path = projectname.value + '/openlayers_updated.html'

append_js_to_html(js_file_path, html_file_path, new_htmlfile_path)

File created: test_PAL11/geojson.js
File updated: test_PAL11/openlayers_updated.html


In [ ]:
#TODO: check default zoom level and remove title and other not needed elements from the html file